In [ ]:
import sys
sys.path.append('../../data/automation_scripts_data/')
from automation_cv_data_processing import run_processing_data_cv
from automation_job_data_processing import run_processing_data_job
from automation_scripts_model.automation_cv_modelling import run_automation_modelling_cv
from automation_scripts_model.automation_job_modelling import run_automation_modelling_job
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Cargamos txt de cv
raw_cvs_SoftwareDev = open('../../data/raw_data/Data_CV_SoftwareDeveloper.txt',encoding='utf-8').read()
raw_cvs_NetworkEng = open('../../data/raw_data/Data_CV_NetworkEngineer.txt',encoding='utf-8').read()
raw_cvs_IADev= open('../../data/raw_data/Data_CV_IADeveloper.txt',encoding='utf-8').read()

In [3]:
# Cargamos txt de jobs
raw_jobs_SoftwareDev = open('../../data/raw_data/Data_Jobs_SoftwareDeveloper.txt',encoding='utf-8').read()
raw_jobs_NetworkEng = open('../../data/raw_data/Data_Jobs_NetworkEngineer.txt',encoding='utf-8').read()
raw_jobs_IADev= open('../../data/raw_data/Data_Jobs_IADeveloper.txt',encoding='utf-8').read()

In [4]:
# Llamamos al script de procesamiento de datos
df_cv_processed = run_processing_data_cv(raw_cvs_SoftwareDev,raw_cvs_NetworkEng,raw_cvs_IADev)
df_jobs_processed = run_processing_data_job(raw_jobs_SoftwareDev,raw_jobs_NetworkEng,raw_jobs_IADev)

--INICIANDO PROCESAMIENTO--
Analizando CVs y extraccion de categorias...
Fusionando categorias procesadas...
Eliminando valores innecesarios...
Estableciendo id de posicion...
Convirtiendo a Dataframe...
Limpieza inicial de datos...
Limpieza semantica de datos...
Aplicando mejoras...
--FIN DEL PROCESAMIENTO--
--INICIANDO PROCESAMIENTO--
Analizando Jobs y extraccion de categorias...
Fusionando categorias procesadas...
Eliminando valores innecesarios...
Estableciendo id de posicion...
Convirtiendo a Dataframe...
Limpieza inicial de datos...
Limpieza semantica de datos...
Aplicando mejoras...
--FIN DEL PROCESAMIENTO--


In [6]:
df_cv_processed.head()

,id,cv_tokens
0,0,desarrollador full stack experimentado nombre ...
1,1,desarrollador móvil junior nombre ana garcía r...
2,2,desarrollador back end especializado nombre ma...
3,3,desarrollador web front end creativo nombre da...
4,4,desarrollador juegos independiente nombre pedr...


In [7]:
df_jobs_processed.head()

,id,job_tokens
0,0,desarrollador software junior aplicaciones móv...
1,1,ingeniero software full stack tecnologías web ...
2,2,desarrollador software frontend experiencia us...
3,3,desarrollador software backend arquitectura mi...
4,4,ingeniero software juegos desarrollo videojueg...


In [8]:
# Carga modelo y generar embeddings
embeddings_cv, df_cvs = run_automation_modelling_cv(df_cv_processed)
embeddings_job, df_jobs = run_automation_modelling_job(df_jobs_processed)

**Calculo similitud del coseno**

In [8]:
def calculate_cosine(mean_embedding_1,mean_embedding_2):
    similarity_score = cosine_similarity([mean_embedding_1], [mean_embedding_2])[0, 0]
    return similarity_score

In [9]:
def similitud_cv(id_cv, id_job):
    
    # Calcular la media de los embeddings
    mean_embedding_1 = np.mean(embeddings_cv[id_cv], axis=0)
    mean_embedding_2 = np.mean(embeddings_job[id_job], axis=0)

    # Calcular la similitud coseno entre los embeddings
    similarity_score = calculate_cosine(mean_embedding_1,mean_embedding_2)

    # Mostrar en pantalla resultados
    print(f"La similaridad entre el CV {id_cv} y la oferta de trabajo {id_job} es de:", similarity_score)
    print('\n------------')
    print(f'CV con id {id_cv}')
    print(df_cvs['cv_tokens'][id_cv])
    print('\n------------')
    print(f'Oferta con id {id_job}')
    print(df_jobs['job_tokens'][id_job])

In [10]:
similitud_cv(8, 2)

La similaridad entre el CV 8 y la oferta de trabajo 2 es de: 0.6782929

------------
CV con id 8
desarrollador web senior nombre andrés martínez resumen profesional desarrollador web senior 10 años experiencia diseño desarrollo sitios web dinámicos aplicaciones web expertise html5 css3 javascript framework react j vue j demostrada capacidad liderar equipos gestionar proyectos principio fin experiencia laboral desarrollador web senior webtech solution lideré equipo desarrollo implementación plataforma comercio electrónico utilizando react j incrementando ventas 40 educación ingeniería informática universidad autónoma madrid uam aptitude desarrollo web full stack gestión proyectos liderazgo técnico

------------
Oferta con id 2
desarrollador software frontend experiencia usuario ux acerca empleo busca desarrollador software frontend apasionado experiencia usuario unirse equipo posición ofrece oportunidad trabajar creación interface usuario intuitivas atractivas aplicaciones web móviles r

**Extraer en funcion de una oferta de trabajo los cv mas similares**

In [9]:
def mean_embedding_job_id(id_job):
    # Calcular la media de los embeddings
    mean_embedding_job = np.mean(embeddings_job[id_job], axis=0)
    return mean_embedding_job

In [10]:
def id_to_textCV(list_cv_id_simil):
    id_cv = [id_[0] for id_ in list_cv_id_simil]
    text_cv = df_cvs.loc[id_cv, 'cv_tokens']
    return text_cv

In [11]:
def shows_cvs_for_this_job(id_job, threshold):
    # retornamos el embedding de la oferta de trabajo con id dado
    embedding_job=mean_embedding_job_id(id_job)

    # Iteramos sobre todos los cv de nuestros embeddings, almacenando la similitud dada por la oferta pasada por parametro
    list_cosine_cv = []
    for cv in range(len(embeddings_cv)):
        # Calcular la media de los embeddings
        mean_embedding_cv = np.mean(embeddings_cv[cv], axis=0)
        list_cosine_cv.append(cosine_similarity([embedding_job], [mean_embedding_cv])[0, 0])

    # Almacenamos unicamente los cv cuya similitud sea mayor o igual al umbral dado por parametro, ordenado descendentemente
    values_cv_threshold = sorted([[i,value] for i,value in enumerate(list_cosine_cv) if value >= threshold],
                                key=lambda x: x[1], reverse=True)
  
    # Extraemos el texto correspondiente al id de los cv con el umbral retornado
    text_cv = id_to_textCV(values_cv_threshold)
    text_job = df_jobs['job_tokens'][id_job]

    # Pintamos el resultado
    print(f'Para la oferta con id {id_job}:')
    print(text_job)
    print(f'------------\nSe han encontrado {len(values_cv_threshold)} CV que podrian ser aptos para dicha oferta:')
    for i, value in enumerate(text_cv):
        print(f'CV {i+1}: {value}')

In [30]:
shows_cvs_for_this_job(id_job=2,threshold=0.7)# id de la oferta de trabajo/umbral de similitud (0-1)

Para la oferta con id 2:
desarrollador software frontend experiencia usuario ux acerca empleo busca desarrollador software frontend apasionado experiencia usuario unirse equipo posición ofrece oportunidad trabajar creación interface usuario intuitivas atractivas aplicaciones web móviles responsabilidades desarrollar interface usuario responsivas visualmente atractivas utilizando html cs javascript colaborar diseñadores ux ui implementar soluciones mejoren experiencia usuario optimizar rendimiento aplicaciones frontend asegurar compatibilidad diferentes dispositivos navegadores cualificación mínimo 2 años experiencia desarrollo frontend profundo conocimiento html cs javascript experiencia framework modernos react vue j angular ofrecemos proyectos emocionantes enfoque experiencia usuario oportunidades crecimiento profesional desarrollo habilidades ambiente trabajo colaborativo creativo
------------
Se han encontrado 5 CV que podrian ser aptos para dicha oferta:
CV 1: desarrollador web fr